# FOR TESTED MONTH 

In [1]:
import pandas as pd
import MySQLdb
import numpy as np
import time
import sys
import datetime 


from sympy import *
from sympy.solvers import solve
from sympy.parsing.sympy_parser import parse_expr

In [3]:
j = 0
d = 30 #Day
all_p = 0
min_p = 0
all_power = []
sum_power = []
sum_p = []
k= 0

# Print iterations progress
# ราย15 วินาทีเป็น 15นาที แสดงว่ามีข้อมูล 96 ค่าข้อมูลต่อหนึ่งวัน ดังนั้นถ้าต้องการข้อมูลในรายชม.ในหนึ่งวัน จะต้องได้ 24ชั่วโมง = 24 ค่าข้อมูล 
# time_a = pd.date_range('2016-09-01', periods=24*d, freq='60T')
# time_b = pd.date_range('2016-09-01 00:59:59', periods=24*d, freq='60T')

time_a = pd.date_range('2016-10-01', periods=96*d, freq='15T')

time_b = pd.date_range('2016-10-01 00:14:59', periods=96*d, freq='15T')

conn= MySQLdb.connect("localhost","conn","hems")
c = conn.cursor()

####### set databases to empty #########
query_empty = """truncate table watt.watt_month_1""" 
c.execute(query_empty)
print "watt.watt_month_1 are Empty!!! \n"

####### set databases_monthly_fee to empty #########
query_empty = """truncate table watt.unit_per_day_1""" 
c.execute(query_empty)
print "watt.unit_per_day_1 are Empty!!! \n"


## Parametr of biil function
# x = raw_input('What about Ft?')
# Ft = float(x)
Ft = -0.048
t1 = 3.2484
t2 = 4.2218
t3 = 4.4217
ser = 38.22


def bill(allpow):    
    ## Fundamental parameter
    if allpow == 0:
        bath = 0
    elif 1 <= allpow <= 150:
        bath = allpow*t1
    elif  151 <= allpow <= 400:
        power = allpow-150
        bath_1 = 150*t1
        bath_2 = power*t2
        bath = bath_1+bath_2
    else:
        power = allpow-150
        bath_1 = 150*t1
        power = power-250
        bath_2 = 250*t2
        bath_3 = power*t3
        bath = bath_1+bath_2+bath_3   
    service_price = 38.22  #permonth (fixed)
   
    ## FT -33.2 satang/unit
    FT_price = allpow*Ft

    ## VAT 7%
    VAT_price = ((service_price+bath)+FT_price)*0.07

    bill = bath+service_price+FT_price+VAT_price

    return round(bill,2)
    

while (j <= (96*d)-1) :
    query = "SELECT Timestamp,watt FROM data_timeline.load_MONTH_1 WHERE Timestamp >= '%s'   AND TimeStamp <= '%s'; "% (time_a[j],time_b[j])
#     print query
    df = pd.read_sql(query, conn)
#     df = df[df.watt != "----"]
#     df = df.reset_index(drop=True)
    df['watt'] = df['watt'].astype(float)
    
    if len(df) == 0 :
        min_p = 0
    else :
        for i in range(len(df)):
            min_p += df.watt[i]  
        min_p = min_p/len(df)/4  ##ข้อมูลพลังงานราย ชม  
    all_p += min_p
    print time_b[j] , "-->" ,float(min_p/1000) ," KW/Hr. that equal to", bill(min_p/1000),"bath -->> equal to",bill(all_p/1000),"bath"  
    c.execute("INSERT INTO watt.watt_month_1 (Timestamp, watt) VALUES ( %s, %s)",(time_b[j],min_p))
    conn.commit()


### คำนวณพลังงานแต่ละวัน โดยการรับข้อมูลมาทั้งหมด ### 
    all_power.append(min_p/1000)
    df1 = pd.DataFrame(all_power)
    while ( k < (len(df1)/96) ) : 
        print """\n. . . . . . . . . . . . . . . . . . . .END DAY  . . . . . . . . . . . .. . . . . . . . . . . . . . .\n"""
#         print (k*24),"to",(k+1)*24
        sum_p =  df1[(k*96):((k+1)*96)].sum()
        sum_power.append(sum_p)
        df2 = pd.DataFrame(sum_power)  
        df2.columns = ['kW/day']
        print df2
        print "Power consuming between" ,time_a[k*96], "to" ,time_b[(k+1)*96-1], "equal to",df2.iloc[k]['kW/day']," kW. -->> equal to",bill(float(df2.iloc[k])),"bath"
        print """///////////////////////////////////////////////////////////////////////////////////////////////////////////////\n"""
        c.execute("INSERT INTO watt.unit_per_day_1 (Timestamp, unit) VALUES ( %s, %s)",(time_b[(k+1)*96-1],df2.iloc[k]['kW/day']))
        conn.commit()
        k +=1 
        
    min_p = 0
    j += 1
    
print(".DONE!")
print """///////////////////////////////////////////////////////////////////////////////////////////////////////////////\n"""
print "Power consuming between" ,time_a[0], "to" ,time_b[j-1], "equal to"  ,all_p/1000 ," kW/Hr = UNIT that equal to",bill(all_p/1000),"bath"   ##พลังงานรวมทั้งหมด

 watt.watt_month_1 are Empty!!! 

watt.unit_per_day_1 are Empty!!! 

2016-10-01 00:14:59 --> 0.0342875  KW/Hr. that equal to -200.73 bath -->> equal to -200.73 bath
2016-10-01 00:29:59 --> 0.0329875  KW/Hr. that equal to -200.74 bath -->> equal to -200.58 bath
2016-10-01 00:44:59 --> 0.029725  KW/Hr. that equal to -200.75 bath -->> equal to -200.44 bath
2016-10-01 00:59:59 --> 0.0333958333333  KW/Hr. that equal to -200.74 bath -->> equal to -200.28 bath
2016-10-01 01:14:59 --> 0.029625  KW/Hr. that equal to -200.75 bath -->> equal to -200.14 bath
2016-10-01 01:29:59 --> 0.0291041666667  KW/Hr. that equal to -200.76 bath -->> equal to -200.01 bath
2016-10-01 01:44:59 --> 0.0330416666667  KW/Hr. that equal to -200.74 bath -->> equal to -199.85 bath
2016-10-01 01:59:59 --> 0.0283208333333  KW/Hr. that equal to -200.76 bath -->> equal to -199.72 bath
2016-10-01 02:14:59 --> 0.0268208333333  KW/Hr. that equal to -200.77 bath -->> equal to -199.59 bath
2016-10-01 02:29:59 --> 0.0275916666667

In [4]:
368.6/30

12.286666666666667